In [4]:
import tensorflow as tf
print(tf.__version__)
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import pandas as pd
import re
import numpy as np
import os
import io
import time

2.6.0


In [5]:
# SQUAD DEV SET ANALYSIS 

In [3]:
dev = pd.read_csv("/content/drive/MyDrive/AutomatedQuestionGeneration/Topic_Bifurcated_dev.csv")

FileNotFoundError: ignored

In [ ]:
dev.drop_duplicates(inplace=True)

In [ ]:
dev.head()

In [ ]:
dev = dev.drop_duplicates(subset = ['context'],keep = 'first').reset_index(drop = True)

In [ ]:
dev.head()

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Colab_data/Topic_Bifurcated_SQUAD1.csv")
df = df.drop_duplicates(subset = ['given_context', 'reference_context'],keep = 'first').reset_index(drop = True)
# df = df[:100]
df.head()

,index,question,context,text,answer_start,c_id,given_context,reference_context
0,56be85543aeaaa14008c9063,When did Beyonce start becoming popular?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,in the late 1990s,269.0,0,"Born and raised in Houston, Texas, she perform...",Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
1,56be85543aeaaa14008c9065,What areas did Beyonce compete in when she was...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,singing and dancing,207.0,0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"Born and raised in Houston, Texas, she perform..."
2,56bf6b0f3aeaaa14008c9605,Who managed the Destiny's Child group?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Mathew Knowles,360.0,0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"Managed by her father, Mathew Knowles, the gro..."
3,56be86cf3aeaaa14008c9076,"After her second solo album, what other entert...",Following the disbandment of Destiny's Child i...,acting,207.0,1,Following the disbandment of Destiny's Child i...,"Beyoncé also ventured into acting, with a Gold..."
4,56bf6e823aeaaa14008c962a,Which album was darker in tone from her previo...,Following the disbandment of Destiny's Child i...,Beyoncé,180.0,1,"Beyoncé also ventured into acting, with a Gold...",Following the disbandment of Destiny's Child i...





# Load data set




In [9]:
# Train Set
reference_context = df["reference_context"].apply(lambda x: x).tolist()
given_context = df["given_context"].apply(lambda x: x).tolist()
target_ques = df["question"].apply(lambda x: x).tolist()

In [10]:
# Test Set
reference_context = dev["reference_context"].apply(lambda x: x).tolist()
given_context = dev["given_context"].apply(lambda x: x).tolist()
target_ques = dev["question"].apply(lambda x: x).tolist()

NameError: ignored

*   Clean the sentences by removing special characters.
*   Add a start and end token to each sentence.
*   Create a word index and reverse word index (dictionaries mapping from word → id and id → word).
*   Pad each sentence to a maximum length.

In [11]:
# Converts the unicode file to ascii
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w):
  w = unicode_to_ascii(str(w).lower().strip())

  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",","¿")
  w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

  # remove extra space
  w = w.strip()

  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  w = '<start> ' + w + ' <end>'
  return w

In [12]:
en_sentence = u"Beyonce's last record was called Lemonade"
sp_sentence = u"Beyonce was born in 1950"
print(preprocess_sentence(en_sentence))
print(preprocess_sentence(sp_sentence))
print(preprocess_sentence(sp_sentence).encode("UTF-8"))

<start> beyonce s last record was called lemonade <end>
<start> beyonce was born in <end>
b'<start> beyonce was born in <end>'


In [13]:
reference_context[:1], given_context[:1]

(['Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy". '],
 ["Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. "])

In [14]:
# def create_dataset_squad(context_param, given_param, target_ques):
  # reference_context = []
  # given_context = []
  # question = []
  # for c,g,q in zip(context_param,given_param, target_ques):
    # reference_context.append(preprocess_sentence(str(c)))
    # given_context.append(preprocess_sentence(str(g)))
    # question.append(preprocess_sentence(q))
  # return tuple(reference_context), tuple(given_context), tuple(question)

# r, g, q = create_dataset_squad(reference_context, given_context, target_ques)
print(r[-1])
print(g[-1])
print(q[-1])
print(len(r), len(q))

<start> this activity has been further enhanced by establishing formal relationships with other cities motsumoto city of japan , rochester of the usa , yangon formerly rangoon of myanmar , xi an of the people s republic of china , minsk of belarus , and pyongyang of the democratic republic of korea . kmc s constant endeavor is to enhance its interaction with saarc countries , other international agencies and many other major cities of the world to achieve better urban management and developmental programs for kathmandu . <end>
<start> kathmandu metropolitan city kmc , in order to promote international relations has established an international relations secretariat irc . kmc s first international relationship was established in with the city of eugene , oregon , united states . <end>
<start> what was yangon previously known as ? <end>
43275 43275


In [40]:
# Tokenize the sentence into list of words(integers) and pad the sequence to the same length
# def tokenize(lang):
#   lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
#       filters='')
#   lang_tokenizer.fit_on_texts(lang)

#   tensor = lang_tokenizer.texts_to_sequences(lang)

#   tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
#                                                          padding='post')
#   return tensor, lang_tokenizer

In [22]:
df = df[:100]

In [41]:
# def load_dataset(reference, given, target):
#   # creating cleaned input, output pairs
#   #targ_lang, inp_lang = create_dataset(path, num_examples)

#   reference_tensor, reference_tokenizer = tokenize(reference)
#   given_tensor, given_tokenizer = tokenize(given)
#   target_tensor, targ_lang_tokenizer = tokenize(target)

#   return reference_tensor, given_tensor, target_tensor, reference_tokenizer, given_tokenizer, targ_lang_tokenizer

In [42]:
# # Try experimenting with the size of that dataset
# #num_examples = 30000
# # input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(c,q)
# reference_tensor, given_tensor, target_tensor, reference_tokenizer, given_tokenizer, targ_tokenizer = load_dataset(r, g, q)

# # Calculate max_length of the target tensors
# max_length_targ, max_length_reference, max_length_given = target_tensor.shape[1], reference_tensor.shape[1], given_tensor.shape[1]
# print(max_length_targ, max_length_reference, max_length_given)

In [25]:
# train = zip(reference_tensor, given_tensor)
# train = pd.DataFrame(data=train, index=None)
# train.head()

In [43]:
context = df["context"].apply(lambda x: x).tolist()
target_ques = df["question"].apply(lambda x: x).tolist()
def create_dataset_squad(context_param,target_ques):
  context = []
  question = []
  for c,q in zip(context_param,target_ques):
    context.append(preprocess_sentence(c))
    question.append(preprocess_sentence(q))
  return context, question

c, q = create_dataset_squad(context,target_ques)
print(c[-1])
print(q[-1])
print(len(c), len(q))
# Tokenize the sentence into list of words(integers) and pad the sequence to the same length
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')
  return tensor, lang_tokenizer
def load_dataset(input, target):
  # creating cleaned input, output pairs
  #targ_lang, inp_lang = create_dataset(path, num_examples)

  input_tensor, inp_lang_tokenizer = tokenize(input)
  target_tensor, targ_lang_tokenizer = tokenize(target)

  return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer
# Try experimenting with the size of that dataset
#num_examples = 30000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(c,q)

# Calculate max_length of the target tensors
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]
print(max_length_targ, max_length_inp)
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))
print(input_tensor_train[1])
print()
print(target_tensor_train[1])

<start> beyonce and husband jay z are friends with president barack obama and first lady michelle obama . she performed america the beautiful at the presidential inauguration , as well as at last during the first inaugural dance at the neighborhood ball two days later . beyonce and jay z held a fundraiser at the latter s club in manhattan for obama s presidential campaign which raised million . beyonce uploaded pictures of her paper ballot on tumblr , confirming she had voted in support for the democratic party and to encourage others to do so . she also performed the american national anthem at his second inauguration , singing along with a pre recorded track . she publicly endorsed same sex marriage on march , , after the supreme court debate on california s proposition . in july , beyonce and jay z attended a rally in response to the acquittal of george zimmerman for the shooting of trayvon martin . <end>
<start> how much did beyonce raise for obama at the club ? <end>
100 100
21 35

In [49]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 16
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
steps_per_epoch_val = len(input_tensor_val)//BATCH_SIZE
embedding_dim = 256  # for word embedding
units = 1024  # dimensionality of the output space of RNN
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
validation_dataset = tf.data.Dataset.from_tensor_slices((input_tensor_val, target_tensor_val)).shuffle(BUFFER_SIZE)
validation_dataset = validation_dataset.batch(BATCH_SIZE, drop_remainder=True)

example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([16, 358]), TensorShape([16, 21]))

In [19]:

# Creating training and validation sets using an 80-20 split
# input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(train, target_tensor, test_size=0.2)
# reference_tensor_train, reference_tensor_val, given_tensor_train, given_tensor_val, target_tensor_train, target_tensor_val = train_test_split(reference_tensor, given_tensor, target_tensor,random_state = 101)

# Show length
# print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))
# print(input_tensor_train[1])
# print()
# print(target_tensor_train[1])

In [20]:
# input_tensor_train
# reference_tensor_train = input_tensor_train[0].values
# given_tensor_train = input_tensor_train[1].values
# 
# reference_tensor_val = input_tensor_val[0].values
# given_tensor_val = input_tensor_val[1].values

In [46]:
# # Configuration 
# BUFFER_SIZE = len(reference_tensor_train)
# BATCH_SIZE = 16
# steps_per_epoch = len(reference_tensor_train)//BATCH_SIZE
# steps_per_epoch_val = len(reference_tensor_val)//BATCH_SIZE
# embedding_dim = 256  # for word embedding
# units = 1024  # dimensionality of the output space of RNN
# vocab_inp_size = len(given_tokenizer.word_index) + len(reference_tokenizer.word_index) + 1 

# vocab_tar_size = len(targ_tokenizer.word_index) + 1

# dataset = tf.data.Dataset.from_tensor_slices((reference_tensor_train, given_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
# dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
# validation_dataset = tf.data.Dataset.from_tensor_slices((reference_tensor_val, given_tensor_val, target_tensor_val)).shuffle(BUFFER_SIZE)
# validation_dataset = validation_dataset.batch(BATCH_SIZE, drop_remainder=True)

# example_reference_batch, example_given_batch, example_target_batch = next(iter(dataset))
# example_reference_batch.shape, example_given_batch.shape, example_target_batch.shape

# Seq2seq model: Two Encoders and DecoderWithDualAttention

In [47]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.vocab_size = vocab_size
    self.embedding_dim = embedding_dim
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.LSTM = tf.keras.layers.LSTM(self.enc_units,
                                   return_sequences=True,  # Whether to return the last output in the output sequence, or the full sequence. 
                                   return_state=True,  # Whether to return the last state in addition to the output.
                                   recurrent_initializer='glorot_uniform')

  
  def get_config(self):
        config = super(Encoder, self).get_config().copy()
        #config = {}
        config.update({
            'batch_sz': self.batch_sz,
            'enc_units': self.enc_units,
            # 'embedding': self.embedding,
            # 'LSTM': self.LSTM,
            'vocab_size': self.vocab_size,
            'embedding_dim': self.embedding_dim
        })
        return config 

  
  def call(self, x, hidden):
    x = self.embedding(x)
    output, hidden_h, hidden_c = self.LSTM(x, initial_state = hidden)
    return output, [hidden_h,hidden_c]

  @tf.function
  def initialize_hidden_state(self):
    init_state = [tf.zeros([self.batch_sz, self.enc_units]) for i in range(2)]
    return init_state

In [48]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
#sample_hidden = [tf.zeros([1,1024]) for i in range(2)]
sample_output, sample_hidden= encoder(example_reference_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden[0].shape))
print ('Encoder Carry state shape: (batch size, units) {}'.format(sample_hidden[1].shape))

Encoder output shape: (batch size, sequence length, units) (16, 452, 1024)
Encoder Hidden state shape: (batch size, units) (16, 1024)
Encoder Carry state shape: (batch size, units) (16, 1024)


# Additive attention

![alt text](https://i.ibb.co/BqDYNP1/additive.jpg)

In [29]:
#tf.keras.layers.Layer

In [30]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units, **kwargs):
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)
    self.units = units
    super(BahdanauAttention, self).__init__(**kwargs)

  
  def get_config(self):
        config = super(BahdanauAttention, self).get_config().copy()
        #config = {}
        config.update({
            'units': self.units,   
        })
        return config
  
  
  
  def call(self, query, values):
    # query hidden state shape == (batch_size, hidden size)
    # query_with_time_axis shape == (batch_size, 1, hidden size)
    # values shape == (batch_size, max_len, hidden size)

    # print("Query : ,",query)
    
    query_with_time_axis = tf.expand_dims(query, 1)
    # print("Query with time axis: ,",query_with_time_axis.shape)
    # query_with_time_axis = query
    # values = query_with_time_axis
    # print("Values: ,",values.shape)
    # print(values.shape)
    # print(query_with_time_axis.shape)

    # (64, 358, 1024) ->values.shape
    # (64, 1, 1024) -> query_with_time_axis.shape

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(query_with_time_axis)))
    # print("W1(Values): ", self.W1(values).shape)
    # print("W2(Query_with_time_axis): ", self.W2(query_with_time_axis).shape)
    # print("Score: ", score.shape)
    # (64, 358, 1024) -> self.W1(values).shape
    # (64, 1, 1024) -> self.W2(query_with_time_axis).shape
    # (64, 358, 1) -> score.shape

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    # print(context_vector.shape)
    # (64, 358, 1024)

    context_vector = tf.reduce_sum(context_vector, axis=1)
    # print(context_vector.shape)
    # (64, 1024)
    return context_vector, attention_weights

In [69]:
class DecoderWithDualAttention(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz, attention_layer1 = None, attention_layer2 = None):
    super(DecoderWithDualAttention, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.LSTM = tf.keras.layers.LSTM(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)
    self.vocab_size = vocab_size
    self.embedding_dim = embedding_dim
    self.Wc = tf.keras.layers.Dense(dec_units, activation=tf.math.tanh,
                                    use_bias=False)
    # used for attention
    self.attention1 = attention_layer1
    self.attention2 = attention_layer2

  
  
  def get_config(self):
        config = super(DecoderWithDualAttention, self).get_config().copy()
        #config = {}
        config.update({
            'batch_sz': self.batch_sz,
            'dec_units': self.dec_units,
            # 'embedding': self.embedding,
            # 'LSTM': self.LSTM,
            # 'fc' : self.fc,
            'attention_layer1':self.attention1,
            'attention_layer2':self.attention2,
            'vocab_size': self.vocab_size,
            'embedding_dim': self.embedding_dim
        })
        return config
    
  
  
  
  def call(self, x, hidden1, enc_output1):
    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)
    # y = x
    attention_weights1 = None
    output1, state_h1, state_c1 = self.LSTM(x, initial_state = hidden1)
    print("output1:", output1.shape)
    print("state_h1:", state_h1.shape)
    # Attention 1 is used to associate the target word embedding with the reference context embedding (attended)
    if self.attention1:
      # enc_output shape == (batch_size, max_length, hidden_size)
      context_vector1, attention_weights1 = self.attention1(output1, enc_output1)
      print("context_vector1:", context_vector1.shape)
      # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
      # output1_context_vector1 = tf.concat([context_vector1, output1], axis=-1)
      output1_context_vector1 = tf.concat([context_vector1, output1], axis=-1)
      output1_context_vector1 = tf.transpose(ooutput1_context_vector1, perm = [0, 2, 1])
      # x = tf.concat([context_vector1, x], axis=-1)
      print("output1_context_vector1:", output1_context_vector1.shape)
    # attention_weights2 = None
    
    # Attention 2 is used to associate the target word embedding with the given context embedding (attended)
    # if self.attention2:
    #   # enc_output shape == (batch_size, max_length, hidden_size)
    #   context_vector2, attention_weights2 = self.attention2(hidden2[0], enc_output2)
    #   # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    #   y = tf.concat([tf.expand_dims(context_vector2, 1), y], axis=-1)
    #   # y = tf.concat([context_vector2, y], axis=-1)
  
    attention_vector = self.Wc(output1_context_vector1)
    print("attention_vector:", attention_vector.shape)
    # passing the concatenated vector to the LSTM
    # output2, state_h2, state_c2= self.LSTM(y, initial_state = hidden2)
    # output shape == (batch_size * 1, hidden_size)
    # output1 = tf.reshape(output1, (-1, output1.shape[2]))
    # output2 = tf.reshape(output2, (-1, output2.shape[2]))

    # output = tf.concat([output1, output2], axis = 1)
    
    # output shape == (batch_size, vocab)
    final_output = self.fc(attention_vector)
    
    # return final_output, [state_h1,state_c1], [state_h2,state_c2], attention_weights1, attention_weights2
    return final_output, [state_h1,state_c1], [1,1], attention_weights1, attention_weights1

In [70]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0)) 
  loss_ = loss_object(real, pred)
  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask
  return tf.reduce_mean(loss_)

In [71]:
print(loss_object([1,2],[[0,0.6,0.3,0.1],[0,0.6,0.3,0.1]]))
print(loss_function([1,2],[[0,0.6,0.3,0.1],[0,0.6,0.3,0.1]]))

tf.Tensor([1.063386  1.3633859], shape=(2,), dtype=float32)
tf.Tensor(1.2133859, shape=(), dtype=float32)


In [72]:
optimizer = tf.keras.optimizers.Adam()

def get_train_step_func():

  @tf.function
  def train_step(inp, targ, enc_hidden1, encoder1, decoder):
    loss = 0

    with tf.GradientTape() as tape: # for automatic differentiation
      enc_output1, enc_hidden1 = encoder1(inp, enc_hidden1)
      # enc_output2, enc_hidden2 = encoder2(aux, enc_hidden2)

      # Initialising decoder hidden states with the corresponding hidden states of the encoder (one for given context encoder and the other for reference context encoder)
      dec_hidden1 = enc_hidden1
      # dec_hidden2 = enc_hidden2
      # enc_hidden -> 

      # Check whether we should concatenate dec_hidden and dec_hidden2 or keep it separate
      # dec_hidden2 = enc_hidden2

      dec_input = tf.expand_dims([targ_tokenizer.word_index['<start>']] * BATCH_SIZE, 1)

      # Teacher forcing - feeding the target as the next input
      for t in range(1, targ.shape[1]):
        # print("Iteration: ",t)
        # passing enc_output to the decoder
        # print("EncOutput1 before: ",enc_output1)
        predictions, dec_hidden1, dec_hidden2, _, _ = decoder(dec_input, dec_hidden1, enc_output1)
        # print("EncOutput1 after: ",enc_output1)
        
        # x, y, hidden1, hidden2, enc_output1, enc_output2

        loss += loss_function(targ[:, t], predictions)

        # using teacher forcing
        dec_input = tf.expand_dims(targ[:, t], 1)

    batch_loss = (loss / int(targ.shape[1]))

    variables = encoder1.trainable_variables + decoder.trainable_variables

    gradients = tape.gradient(loss, variables)

    optimizer.apply_gradients(zip(gradients, variables))

    return batch_loss
    
  return train_step

In [73]:
def caculate_validation_loss(inp, targ, enc_hidden1, encoder1, decoder):
  loss = 0
  enc_output1, enc_hidden1 = encoder1(inp, enc_hidden1)
  # enc_output2, enc_hidden2 = encoder2(aux, enc_hidden1)
  dec_hidden1 = enc_hidden1
  # dec_hidden2 = enc_hidden2
  dec_input = tf.expand_dims([targ_tokenizer.word_index['<start>']] * BATCH_SIZE, 1)

  # Teacher forcing - feeding the target as the next input
  # for t in range(1, targ.shape[1]):
  #   predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
  #   loss += loss_function(targ[:, t], predictions)
  #   dec_input = tf.expand_dims(targ[:, t], 1)

  # loss = loss / int(targ.shape[1])
  # return loss

  for t in range(1, targ.shape[1]):
        # passing enc_output to the decoder
        predictions, dec_hidden1, dec_hidden2, _, _ = decoder(dec_input, dec_hidden1, enc_output1)
        # x, y, hidden1, hidden2, enc_output1, enc_output2

        loss += loss_function(targ[:, t], predictions)

        # using teacher forcing
        dec_input = tf.expand_dims(targ[:, t], 1)

  loss = loss / int(targ.shape[1])
  return loss

In [74]:
 def training_seq2seq(epochs, attention1):
  encoder1 = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
  # encoder2 = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
  decoder = DecoderWithDualAttention(vocab_tar_size, embedding_dim, units, BATCH_SIZE, attention1, attention2)
  train_step_func = get_train_step_func()
  training_loss = []
  validation_loss = []

  for epoch in range(epochs):
    print()
    print(epoch)
    start = time.time()
    enc_hidden1 = encoder1.initialize_hidden_state()
    # enc_hidden2 = encoder2.initialize_hidden_state()
    total_loss = 0

    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
      print(batch)
      batch_loss = train_step_func(inp, targ, enc_hidden1, encoder1, decoder)
      # inp, targ, aux, enc_hidden, enc_hidden2, encoder, encoder2, decoder
      total_loss += batch_loss

      if batch % 100 == 0:
        print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1, batch, batch_loss))
        
    enc_hidden1 = encoder1.initialize_hidden_state()
    # enc_hidden2 = encoder2.initialize_hidden_state()
    total_val_loss = 0
    for (batch, (inp, targ)) in enumerate(validation_dataset.take(steps_per_epoch)):
      val_loss = caculate_validation_loss(inp, targ, enc_hidden1, encoder1, decoder)
      total_val_loss += val_loss

    training_loss.append(total_loss / steps_per_epoch)
    validation_loss.append(total_val_loss / steps_per_epoch_val)
    print('Epoch {} Loss {:.4f} Validation Loss {:.4f}'.format(epoch + 1,
                                        training_loss[-1], validation_loss[-1]))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))
  return encoder1, 1, decoder, training_loss, validation_loss

In [75]:
# Training Starts Here

In [ ]:
def resume_training_seq2seq(epochs, attention1, attention2):
  encoder1 = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
  encoder2 = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
  decoder = DecoderWithDualAttention(vocab_tar_size, embedding_dim, units, BATCH_SIZE, attention1, attention2)
  encoder1.load_weights("/content/drive/MyDrive/AutomatedQuestionGeneration/trained-model/DualAtttentionLSTMBased/encoder1-10")
  encoder2.load_weights("/content/drive/MyDrive/AutomatedQuestionGeneration/trained-model/DualAtttentionLSTMBased/encoder2-10")
  decoder.load_weights("/content/drive/MyDrive/AutomatedQuestionGeneration/trained-model/DualAtttentionLSTMBased/decoder10")
  print("\n\nLoading done\n\n")
  train_step_func = get_train_step_func()
  training_loss = []
  validation_loss = []

  for epoch in range(epochs):
    print()
    print(epoch)
    start = time.time()
    enc_hidden1 = encoder1.initialize_hidden_state()
    enc_hidden2 = encoder2.initialize_hidden_state()
    total_loss = 0

    for (batch, (aux, inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
      print(batch)
      batch_loss = train_step_func(inp, targ, aux, enc_hidden1, enc_hidden2, encoder1, encoder2, decoder)
      # inp, targ, aux, enc_hidden, enc_hidden2, encoder, encoder2, decoder
      total_loss += batch_loss
      print("\n\nBatch done\n\n")

      if batch % 100 == 0:
        print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1, batch, batch_loss))
        
    enc_hidden1 = encoder1.initialize_hidden_state()
    enc_hidden2 = encoder2.initialize_hidden_state()
    total_val_loss = 0
    for (batch, (aux, inp, targ)) in enumerate(validation_dataset.take(steps_per_epoch)):
      val_loss = caculate_validation_loss(inp, targ, aux, enc_hidden1, enc_hidden2, encoder1, encoder2, decoder)
      total_val_loss += val_loss

    training_loss.append(total_loss / steps_per_epoch)
    validation_loss.append(total_val_loss / steps_per_epoch_val)
    print('Epoch {} Loss {:.4f} Validation Loss {:.4f}'.format(epoch + 1,
                                        training_loss[-1], validation_loss[-1]))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))
  return encoder1, encoder2, decoder, training_loss, validation_loss

## Training seq2seq with Bahdanau attention

In [76]:
epochs = 1

attention1 = BahdanauAttention(units)
# attention2 = BahdanauAttention(units)
print("Running seq2seq model with Bahdanau attention")
encoder1_bah, encoder2_bah2, decoder_bah, training_loss, validation_loss = training_seq2seq(epochs, attention1)


Running seq2seq model with Bahdanau attention

0
0
output1: (16, 1, 1024)
state_h1: (16, 1024)
context_vector1: (16, 358, 1024)


ValueError: ignored

In [67]:
a = tf.ones((16, 358, 1024))
b = tf.zeros((16, 1, 1024))

In [68]:
tf.concat([a, b], axis=-1)

InvalidArgumentError: ignored

In [ ]:
tf.keras.models.save_model(encoder1_bah,"/content/drive/MyDrive/AutomatedQuestionGeneration/trained-model/TF2.6saved/lstm_output_dual/encoder1-final")
tf.keras.models.save_model(encoder2_bah,"/content/drive/MyDrive/AutomatedQuestionGeneration/trained-model/TF2.6saved/lstm_output_dual/encoder2-final")
tf.keras.models.save_model(decoder_bah,"/content/drive/MyDrive/AutomatedQuestionGeneration/trained-model/TF2.6saved/lstm_output_dual/decoder-final")

In [ ]:
# Loading Trained Model
attention1 = BahdanauAttention(units)
attention2 = BahdanauAttention(units)
encoder1 = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
encoder2 = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
decoder = DecoderWithDualAttention(vocab_tar_size, embedding_dim, units, BATCH_SIZE, attention1, attention2)

In [ ]:
encoder1 = tf.keras.models.load_model("/content/drive/MyDrive/AutomatedQuestionGeneration/trained-model/encoder1-final/")
encoder2 = tf.keras.models.load_model("/content/drive/MyDrive/AutomatedQuestionGeneration/trained-model/encoder2-final")
decoder = tf.keras.models.load_model("/content/drive/MyDrive/AutomatedQuestionGeneration/trained-model/decoder-final/")

In [ ]:
def translate(g_context, r_context, encoder1, encoder2, decoder):
  attention_plot1 = np.zeros((max_length_targ, max_length_given))
  attention_plot2 = np.zeros((max_length_targ, max_length_reference))

  g_context = preprocess_sentence(g_context)

  given = [given_tokenizer.word_index[i] for i in g_context.split(' ')]
  given = tf.keras.preprocessing.sequence.pad_sequences([given],
                                                         maxlen=max_length_given,
                                                         padding='post')
  # given = tf.convert_to_tensor(given)
  given = tf.reshape(tf.convert_to_tensor(given),shape = [-1,max_length_given])

  
  r_context = preprocess_sentence(r_context)
  
  ref = [reference_tokenizer.word_index[i] for i in r_context.split(' ')]
  ref = tf.keras.preprocessing.sequence.pad_sequences([ref],
                                                         maxlen=max_length_reference,
                                                         padding='post')
  # ref = tf.convert_to_tensor(ref)
  ref = tf.reshape(tf.convert_to_tensor(ref),shape = [-1,max_length_reference])


  result = ''

  hidden = [tf.zeros((1, units)) for i in range(2)]
  # hidden = encoder1.initialize_hidden_state()
  enc_out1, enc_hidden1 = encoder1(given, hidden)
  enc_out2, enc_hidden2 = encoder2(ref, hidden)

  dec_hidden1 = enc_hidden1
  dec_hidden2 = enc_hidden2
  dec_input = tf.expand_dims([targ_tokenizer.word_index['<start>']], 0)

  for t in range(max_length_targ):
    # predictions, dec_hidden, attention_weights = decoder(dec_input,
    #                                                      dec_hidden,
    #                                                      enc_out)

    predictions, dec_hidden1, dec_hidden2, _, _ = decoder(dec_input, dec_hidden1, dec_hidden2, enc_out1, enc_out2)

    predicted_id = tf.argmax(predictions[0]).numpy() 
    # print(predicted_id)

    result += targ_tokenizer.index_word[predicted_id] + ' '
    # print(result)

    # until the predicted word is <end>.
    if targ_tokenizer.index_word[predicted_id] == '<end>':
      return result

    # the predicted ID is fed back into the model, no teacher forcing.
    dec_input = tf.expand_dims([predicted_id], 0)

  return result

In [ ]:
dev.head()

,title,context,question,answer_index,answer,given_context,reference_context
0,Super_Bowl_50,Super Bowl 50 was an American football game to...,Which NFL team represented the AFC at Super Bo...,177,Denver Broncos,The American Football Conference (AFC) champio...,Super Bowl 50 was an American football game to...
1,Super_Bowl_50,Super Bowl 50 was an American football game to...,Which NFL team represented the AFC at Super Bo...,177,Denver Broncos,Super Bowl 50 was an American football game to...,The American Football Conference (AFC) champio...
3,Super_Bowl_50,Super Bowl 50 was an American football game to...,Which NFL team represented the NFC at Super Bo...,249,Carolina Panthers,Super Bowl 50 was an American football game to...,The American Football Conference (AFC) champio...
6,Super_Bowl_50,Super Bowl 50 was an American football game to...,Where did Super Bowl 50 take place?,403,"Santa Clara, California",Super Bowl 50 was an American football game to...,"The game was played on February 7, 2016, at Le..."
7,Super_Bowl_50,Super Bowl 50 was an American football game to...,Where did Super Bowl 50 take place?,355,Levi's Stadium,Super Bowl 50 was an American football game to...,"The game was played on February 7, 2016, at Le..."


In [ ]:
translate(dev.iloc[10,5],dev.iloc[10,6], encoder1, encoder2, decoder)

'what was the first year that the cia s budget was disclosed ? <end> '

In [ ]:
dev.iloc[10,5]

"Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. "

In [ ]:
dev.iloc[10,6]

'The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50. '

In [ ]:
dev.iloc[10,2]

'What was the theme of Super Bowl 50?'

In [ ]:
dev.head()

,title,context,question,answer_index,answer,given_context,reference_context
0,Super_Bowl_50,Super Bowl 50 was an American football game to...,Which NFL team represented the AFC at Super Bo...,177,Denver Broncos,The American Football Conference (AFC) champio...,Super Bowl 50 was an American football game to...
1,Super_Bowl_50,Super Bowl 50 was an American football game to...,Which NFL team represented the AFC at Super Bo...,177,Denver Broncos,Super Bowl 50 was an American football game to...,The American Football Conference (AFC) champio...
3,Super_Bowl_50,Super Bowl 50 was an American football game to...,Which NFL team represented the NFC at Super Bo...,249,Carolina Panthers,Super Bowl 50 was an American football game to...,The American Football Conference (AFC) champio...
6,Super_Bowl_50,Super Bowl 50 was an American football game to...,Where did Super Bowl 50 take place?,403,"Santa Clara, California",Super Bowl 50 was an American football game to...,"The game was played on February 7, 2016, at Le..."
7,Super_Bowl_50,Super Bowl 50 was an American football game to...,Where did Super Bowl 50 take place?,355,Levi's Stadium,Super Bowl 50 was an American football game to...,"The game was played on February 7, 2016, at Le..."


In [ ]:
try:
  translate(df.iloc[8578,6],df.iloc[8578,7], encoder1, encoder2, decoder)
except:
  print("Error")

Error


In [ ]:
df.iloc[10000,6]

"There are numerous community and international newspapers locally that cater to the city's ethnic mosaic; such as The Black Chronicle, headquartered in the Eastside, the OK VIETIMES and Oklahoma Chinese Times, located in Asia District, and various Hispanic community publications. "

In [ ]:
df.iloc[10000,7]

'The Campus is the student newspaper at Oklahoma City University. Gay publications include The Gayly Oklahoman. '

In [ ]:
df.iloc[10100,6]

'Postmarital residence among hunter-gatherers tends to be matrilocal, at least initially. Young mothers can enjoy childcare support from their own mothers, who continue living nearby in the same camp. '

In [ ]:
df.head()

In [ ]:
predict_df = pd.read_excel("/content/drive/MyDrive/AutomatedQuestionGeneration/trained-model/LSTMpredictions17861.xlsx").drop("Unnamed: 0",axis = 1)

In [ ]:
predict_df = df[["given_context","reference_context"]]
predict_df.columns = ["given_context","reference_context"]
predict_df["generated_question"] = 'a'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
for i in range(8570,8580):
  try:
    predict_df["generated_question"][i] = translate(predict_df["given_context"][i], predict_df["reference_context"][i],encoder1,encoder2,decoder)
  except:
    predict_df["given_context"][i], predict_df["reference_context"][i] = np.nan,np.nan

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
predict_df[8570:8581].isna().sum()

In [ ]:
predict_df[8570:8581]

,given_context,reference_context,generated_question
8570,According to Y chromosome studies by Sanchez e...,"(2004, 2007), the Somalis are paternally close...",what is the most common surname in the italian...
8571,"(2005), Cruciani et al. Sanchez et al. Accordi...",According to Y chromosome studies by Sanchez e...,what is the most efficient means of transporta...
8572,According to mtDNA studies by Holden (2005) an...,"(2006), a significant proportion of the matern...",what is the most efficient means of transporta...
8573,According to mtDNA studies by Holden (2005) an...,This mitochondrial clade is common among Ethio...,what is the opposite of blackness ? <end>
8574,"(2006), a significant proportion of the matern...",According to mtDNA studies by Holden (2005) an...,what is the most efficient means of transporta...
8575,"(2006), a significant proportion of the matern...",According to mtDNA studies by Holden (2005) an...,what is the most common soundboard shape ? <end>
8576,According to an autosomal DNA study by Hodgson...,"(2014), the Afro-Asiatic languages were likely...",what was the landline growth rate of somalia i...
8577,According to an autosomal DNA study by Hodgson...,"(2014), the Afro-Asiatic languages were likely...",what is the posulated homeland region of the r...
8578,NaN,NaN,a
8579,The history of Islam in Somalia is as old as t...,The early persecuted Muslims fled to various p...,what is the name of the nomad tribes that hunt...


In [ ]:
#Given_Context
df.iloc[10,6]

"Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. "

In [ ]:
#Reference_Context
df.iloc[10,6]

"Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. "

In [ ]:
predict_df.iloc[10,2]

'when did beyonce start becoming a golden globe ? <end> '

In [ ]:
df.iloc[10,1]

'What was the first album Beyoncé released as a solo artist?'

In [ ]:
import math
math.isnan(df["reference_context"][87])

True

In [ ]:
pd.isnull(df)

In [ ]:
squad.iloc[8578,2]

'After August 2008 sites had to be listed on the Open Directory in order to be included. According to Jeff Kaplan of the Internet Archive in November 2010, other sites were still being archived, but more recent captures would become visible only after the next major indexing, an infrequent operation.'

In [ ]:
df.iloc[8578,6]

'According to Mohamoud et al. '

In [ ]:
df.iloc[8578,7]

'(2006): '

In [ ]:
df.head()

,index,question,context,text,answer_start,c_id,given_context,reference_context
0,56be85543aeaaa14008c9063,When did Beyonce start becoming popular?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,in the late 1990s,269.0,0,"Born and raised in Houston, Texas, she perform...",Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
1,56be85543aeaaa14008c9065,What areas did Beyonce compete in when she was...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,singing and dancing,207.0,0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"Born and raised in Houston, Texas, she perform..."
2,56bf6b0f3aeaaa14008c9605,Who managed the Destiny's Child group?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Mathew Knowles,360.0,0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"Managed by her father, Mathew Knowles, the gro..."
3,56be86cf3aeaaa14008c9076,"After her second solo album, what other entert...",Following the disbandment of Destiny's Child i...,acting,207.0,1,Following the disbandment of Destiny's Child i...,"Beyoncé also ventured into acting, with a Gold..."
4,56bf6e823aeaaa14008c962a,Which album was darker in tone from her previo...,Following the disbandment of Destiny's Child i...,Beyoncé,180.0,1,"Beyoncé also ventured into acting, with a Gold...",Following the disbandment of Destiny's Child i...


In [ ]:
newdf = df.drop_duplicates(
  subset = ['context'],
  keep = 'first').reset_index(drop = True)

In [ ]:
newdf.to_excel("/content/drive/MyDrive/AutomatedQuestionGeneration/UniqueContextBifurcated.xlsx")

In [ ]:
predict_df.head(6)

In [ ]:
index = int(input("Enter record index: "))
#Given_Context
print("Given context:\n",predict_df.iloc[index,0])
#Reference_Context
print("\nReference context:\n",predict_df.iloc[index,1])
# Generated Question
print("\nGenerated question: ",predict_df.iloc[index,3])
# Target Question
print("\nTarget question: ",predict_df.iloc[index,2])

Enter record index: 0
Given context:
 Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. 

Reference context:
 Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy". 

Generated question:  when did beyonce start becoming popular ? <end> 

Target question:  When did Beyonce start becoming popular?


In [ ]:
predict_df.iloc[0,1][:146]

'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress.'

In [ ]:
translate(predict_df.iloc[0,0][:189],predict_df.iloc[0,1][:146],encoder1,encoder2,decoder)

'escalation relationships dumbwaiters witon paints paints paints satavahanas satavahanas montevideo luxuries prevailed montevideo prevailed prevailed analogies leto leto leto valens starve complaining repress numbe statistical shots valens vetoes rockerfeller novels novels novels leto novels leto novels leto nassir leto novels leto nassir leto novels leto novels leto nassir leto novels leto novels leto nassir leto synthesized cabildo novels novels leto novels leto '

In [ ]:
1squad = pd.read_excel("/content/drive/MyDrive/AutomatedQuestionGeneration/train.xlsx").drop("Unnamed: 0",axis = 1)

In [ ]:
squad.iloc[298,2]

'On December 13, 2013, Beyoncé unexpectedly released her eponymous fifth studio album on the iTunes Store without any prior announcement or promotion. The album debuted atop the Billboard 200 chart, giving Beyoncé her fifth consecutive number-one album in the US. This made her the first woman in the chart\'s history to have her first five studio albums debut at number one. Beyoncé received critical acclaim and commercial success, selling one million digital copies worldwide in six days; The New York Times noted the album\'s unconventional, unexpected release as significant. Musically an electro-R&B album, it concerns darker themes previously unexplored in her work, such as "bulimia, postnatal depression [and] the fears and insecurities of marriage and motherhood". The single "Drunk in Love", featuring Jay Z, peaked at number two on the Billboard Hot 100 chart. In April 2014, after much speculation in the weeks before, Beyoncé and Jay Z officially announced their On the Run Tour. It ser

In [ ]:
squad.iloc[60893,2]

'In 1992, Madonna had a role in A League of Their Own as Mae Mordabito, a baseball player on an all-women\'s team. She recorded the film\'s theme song, "This Used to Be My Playground", which became a Hot 100 number one hit. The same year, she founded her own entertainment company, Maverick, consisting of a record company (Maverick Records), a film production company (Maverick Films), and associated music publishing, television broadcasting, book publishing and merchandising divisions. The deal was a joint venture with Time Warner and paid Madonna an advance of $60 million. It gave her 20% royalties from the music proceedings, one of the highest rates in the industry, equaled at that time only by Michael Jackson\'s royalty rate established a year earlier with Sony. The first release from the venture was Madonna\'s book, titled Sex. It consisted of sexually provocative and explicit images, photographed by Steven Meisel. The book received strong negative reaction from the media and the ge

In [ ]:
squad.iloc[60893,1]

'What was the name of the erotic thriller that shows scenes of sadomasochism and bondage?'

In [ ]:
translate(df["given_context"][0], df["reference_context"][0],encoder1,encoder2,decoder)

'what was the name of the film that madonna directed ? <end> '

In [ ]:
df["given_context"][0]

"Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. "

In [ ]:
df["reference_context"][0]

'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy". '

In [ ]:
df.head()

In [ ]:
newdf = df.drop_duplicates(
  subset = ['given_context', 'reference_context'],
  keep = 'first').reset_index(drop = True)
  
# print latest dataframe
display(newdf)

In [ ]:
newdf.head(20)

In [ ]:
# BLEU SCORE
from nltk.translate.bleu_score import sentence_bleu
reference = [['this',"is","a","good","day"]]
candidate = [['hey',"hi","bad"],["its","not"]]
score = sentence_bleu(ref, given, weights=(1, 0, 0, 0))
print(score)

1.0


In [ ]:
ref = [x.split() for x in grp.iloc[0,1]]
given = ref[0]

In [ ]:
given

['When', 'did', 'Beyonce', 'start', 'becoming', 'popular?']

In [ ]:
squad = pd.read_excel("/content/drive/MyDrive/AutomatedQuestionGeneration/train.xlsx").drop("Unnamed: 0",axis = 1)

In [ ]:
squad = squad[:100]

In [ ]:
grp = squad.groupby('context',sort=False).agg(new=('question', list)).reset_index()

In [ ]:
def create_reference_dataset_bleu(df):
  # Takes the SQUAD dataset
  # Groups by "context"
  # For every unique contxet, a list of all the questions in the squad dataset is created
  grp = df.groupby('context',sort=False).agg(new=('question', list)).reset_index()
  return grp

In [ ]:
def create_candidate_dataset_bleu(df):
  # Takes the SQUAD dataset
  # Groups by "context"
  # For every unique contxet, a list of all the questions in the squad dataset is created
  grp = df.groupby('context',sort=False).agg(new=('question', list)).reset_index()
  return grp

In [ ]:
# Loading Trained Model
attention1 = BahdanauAttention(units)
attention2 = BahdanauAttention(units)
encoder1 = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
encoder2 = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
decoder = DecoderWithDualAttention(vocab_tar_size, embedding_dim, units, BATCH_SIZE, None, None)
encoder1.load_weights("/content/drive/MyDrive/AutomatedQuestionGeneration/trained-model/DualAtttentionLSTMBased/encoder1-6",)
encoder2.load_weights("/content/drive/MyDrive/AutomatedQuestionGeneration/trained-model/DualAtttentionLSTMBased/encoder2-6")
decoder.load_weights("/content/drive/MyDrive/AutomatedQuestionGeneration/trained-model/DualAtttentionLSTMBased/decoder6")

In [ ]:
def calculate_bleu(df):
  reference = []
  data = create_reference_dataset_bleu(df)
  bleu = 0
  for i in range(len(data[:2])):
    reference = [x.split() for x in grp.iloc[i,1]]
    print(len(data))
    print(len(df))
  return ref

In [ ]:
len(create_dataset_bleu(df))

18877

In [ ]:
calculate_bleu(df)

In [ ]:
grp.head()

,context,new
0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"[When did Beyonce start becoming popular?, Wha..."
1,Following the disbandment of Destiny's Child i...,"[After her second solo album, what other enter..."
2,"A self-described ""modern-day feminist"", Beyonc...","[In her music, what are some recurring element..."
3,"Beyoncé Giselle Knowles was born in Houston, T...",[Beyonce's younger sibling also sang with her ...
4,Beyoncé attended St. Mary's Elementary School ...,"[What town did Beyonce go to school in?, Who w..."


In [ ]:
squad.groupby('context').apply(f).iloc[2][0]

"Beyonce's younger sibling also sang with her in what band?Where did Beyonce get her name from?What race was Beyonce's father?Beyonce's childhood home believed in what religion?Beyonce's father worked as a sales manager for what company?Beyonce's mother worked in what industry?What younger sister of Beyonce also appeared in Destiny's Child?Beyonce is a descendent of what Arcadian leader?What company did Beyoncé's father work for when she was a child?What did Beyoncé's mother own when Beyoncé was a child?What is the name of Beyoncé's younger sister?Beyoncé is a descendant of which Acadian leader?Beyoncé was raised in what religion?"

### Doing it for a simple example batch

In [ ]:
# attention = BahdanauAttention(units)
# encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
# decoder = DecoderWithAttention(vocab_tar_size, embedding_dim, units, BATCH_SIZE, attention)
# loss = 0
# with tf.GradientTape() as tape: # for automatic differentiation
#     sample_hidden = encoder.initialize_hidden_state()
#     enc_output, enc_hidden = encoder(example_input_batch, sample_hidden)

#     dec_hidden = enc_hidden
#     dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)
#     # Teacher forcing - feeding the target as the next input
#     for t in range(1, example_target_batch.shape[1]):
#         # passing enc_output to the decoder
#         predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
#         loss += loss_function(example_target_batch[:, t], predictions)
#         # using teacher forcing
#         dec_input = tf.expand_dims(example_target_batch[:, t], 1)